# Section 2 : Downloading data

## Getting hands on the right library and table

In [37]:
import wrds
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme()

In [38]:
db=wrds.Connection(wrds_username='arthurdhonneur')

Loading library list...
Done


In [39]:
db.list_libraries()

['aha_sample',
 'ahasamp',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'bank_premium_samp',
 'banksamp',
 'block',
 'block_all',
 'boardex_trial',
 'boardsmp',
 'bvd_amadeus_trial',
 'bvd_bvdbankf_trial',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'candid_samp',
 'cboe',
 'cboe_all',
 'cboe_sample',
 'cboesamp',
 'cddsamp',
 'ciqsamp',
 'ciqsamp_capstrct',
 'ciqsamp_common',
 'ciqsamp_keydev',
 'ciqsamp_pplintel',
 'ciqsamp_ratings',
 'ciqsamp_transactions',
 'ciqsamp_transcripts',
 'cisdmsmp',
 'columnar',
 'comp',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global_daily',
 'comp_na_daily_all',
 'comp_segments_hist_daily',
 'compsamp',
 'compsamp_all',
 'compsamp_snapshot',
 'compseg',
 'contrib',
 'contrib_as_filed_financials',
 'contrib_ceo_turnover',
 'contrib_char_returns',
 'contrib_corporate_culture',
 'contrib_general',
 'contrib_global_factor',
 'contrib_intangible_value',
 'contrib_kpss',
 'contrib_liva',
 'crsp',
 'crsp_a_ccm',
 'crsp_a_inde

In [40]:
db.list_tables(library='wrdsapps_windices')


['dwcountryreturns', 'mwcountryreturns', 'wcountryconstituents']

In [41]:
df = db.get_table(library='wrdsapps_windices', table='mwcountryreturns')

df.head()

,fic,date,country,currency,mportret,mportretx
0,AUS,1986-07-31,AUSTRALIA,AUD,-0.0547,-0.056155
1,AUS,1986-08-29,AUSTRALIA,AUD,0.040118,0.0393
2,AUS,1986-09-30,AUSTRALIA,AUD,0.047842,0.045157
3,AUS,1986-10-31,AUSTRALIA,AUD,0.106395,0.097698
4,AUS,1986-11-28,AUSTRALIA,AUD,0.005453,0.00311


In [42]:
#---------------------------------------------
# Value Weighted Index Returns
#---------------------------------------------
Rm=db.raw_sql("""select  date,vwretd from crsp.msi 
                where date>='2002-01-01' and date<='2024-12-31'
                """,date_cols=['date'])
Rm = Rm.rename(columns={'vwretd':'Rm'})

Rm.head()

,date,Rm
0,2002-01-31,-0.015966
1,2002-02-28,-0.0217
2,2002-03-28,0.044698
3,2002-04-30,-0.0496
4,2002-05-31,-0.01051


In [43]:
#---------------------------------------------
# Risk Free Rate 
#---------------------------------------------
Rf=db.raw_sql("""select  mcaldt,tmytm 
           from crsp.tfz_mth_rf            
            where kytreasnox = 2000001 
           and mcaldt>='2002-01-01'
            and mcaldt<='2024-12-31'""", date_cols=['mcaldt'])
Rf['tmytm']=Rf['tmytm']/12/100
Rf=Rf.rename(columns={ "mcaldt": "date","tmytm": "rf"})

Rf.tail()

,date,rf
271,2024-08-30,0.00438
272,2024-09-30,0.003826
273,2024-10-31,0.003752
274,2024-11-29,0.003475
275,2024-12-31,0.003337


## Looking at the fx data


In [45]:
import glob

paths_fx = glob.glob('data/*.csv')

for path in paths_fx:
    curr = path.split('/')[-1].split('.')[0]
    print(f'the currency :{curr}')
    
    df_tamp = pd.read_csv(path)
    print(f'the len of the data :{len(df_tamp)}')
    print('--'*20)

the currency :USD_GBP
the len of the data :276
----------------------------------------
the currency :CHF_USD
the len of the data :276
----------------------------------------
the currency :JPY_USD
the len of the data :276
----------------------------------------
the currency :USD_AUD
the len of the data :276
----------------------------------------
the currency :USD_EUR
the len of the data :276
----------------------------------------


## Working with the right library / table

In [44]:
df = df[(df['date'] >= '2002-01-01') & (df['date'] <= '2024-12-31')]

list_countries = ['AUS', 'FRA', 'DEU', 'JPN', 'CHE','GBR']
df = df[df['fic'].isin(list_countries)]

df.sample(10)

,fic,date,country,currency,mportret,mportretx
342,AUS,2015-01-30,AUSTRALIA,AUD,0.031928,0.031852
1800,CHE,2016-11-30,SWITZERLAND,CHF,0.012799,0.012799
242,AUS,2006-09-29,AUSTRALIA,AUD,0.013369,0.007522
4483,FRA,2006-07-31,FRANCE,EUR,0.009762,0.009089
4699,FRA,2024-07-31,FRANCE,EUR,0.002655,0.001466
3083,DEU,2023-10-31,GERMANY,EUR,-0.045291,-0.045319
4701,FRA,2024-09-30,FRANCE,EUR,0.00874,0.00799
1888,CHE,2024-03-28,SWITZERLAND,CHF,0.032764,0.025482
4537,FRA,2011-01-31,FRANCE,EUR,0.038354,0.038282
241,AUS,2006-08-31,AUSTRALIA,AUD,0.033225,0.025378
